# Evaluation pipeline: classifier baselines

In [ ]:
import torch
from pytorch_pretrained_bert import BertTokenizer,BertForMaskedLM
from sentence_transformers import SentenceTransformer
from itertools import chain
import pandas as pd
import numpy as np
import spacy
import operator
from pathlib import Path
from scipy import stats
from gensim.models.wrappers import FastText
from gensim.models import Word2Vec
from tools import animacy_evaluation,processing
import unidecode
from collections import Counter
import sklearn
import pathlib
import pickle
from tools import processing

In [ ]:
from tools import classifiers

In [ ]:
# ----------------------------
# Select corpus to classify:
classifier_corpus = "stories/" # Options: "stories/" or "machines19thC
testing_corpus = "machines19thC/" # Options: "stories/" or "machines19thC"

abspath = "/Users/mcollardanuy/Documents/githubCode/AtypicalAnimacy/"

In [ ]:
# ----------------------------
# Load data
scenarios = {"stories/" : "all_", "machines19thC/": "animacy_all_"}
classifier_scenario = scenarios[classifier_corpus]
testing_scenario = scenarios[testing_corpus]

dataset_traindf = pd.read_pickle(abspath + "data/" + testing_corpus + testing_scenario + "train.pkl")
dataset_traindf['both_masked'] = dataset_traindf.apply(lambda row: processing.determine_context("maskedSentence", row, "both"), axis=1)
dataset_traindf['both_unmasked'] = dataset_traindf.apply(lambda row: processing.determine_context("currentSentence", row, "both"), axis=1)

dataset_testdf = pd.read_pickle(abspath + "data/" + testing_corpus + testing_scenario + "test.pkl")
dataset_testdf['both_masked'] = dataset_testdf.apply(lambda row: processing.determine_context("maskedSentence", row, "both"), axis=1)
dataset_testdf['both_unmasked'] = dataset_testdf.apply(lambda row: processing.determine_context("currentSentence", row, "both"), axis=1)

In [ ]:
for classifier in ["tfidf_svm", "wemb_svm"]:
    for type_of_training_data in ["targetExpression", "context3wmasked", "context3w", "both_masked", "both_unmasked", "currentSentence", "maskedSentence"]:

        threshold_list = list(np.arange(0, 1.05, 0.1))

        df_results = pd.DataFrame(columns = ['threshold', 'precision', 'recall', 'fscore', 'micro_fscore', 'map'])
        print("\nType of training data:", type_of_training_data)
        print("Classifier:", classifier)
        print("Corpus used to train the classifier:", classifier_corpus)
        print("Corpus used for parameter-tuning and testing:", testing_corpus)

        if not Path(abspath + "experiments/" + testing_corpus + "classifier_" + testing_scenario + "train_" + classifier + "_" + type_of_training_data + ".tsv").exists():

            # -------------------------------------------------
            # Optimal threshold from train:
            for th in threshold_list:
                print("* Tuning parameters.")
                th = float(round(th,2))
                y_pred = classifiers.classify(abspath,classifier,dataset_traindf[type_of_training_data],type_of_training_data,classifier_corpus,classifier_scenario)
                y_true = [x for x in dataset_traindf['animated'].tolist()]

                precision, recall, fscore, micro_fscore,map_ = animacy_evaluation.results(y_true,y_pred,th)
                df_results = df_results.append({'threshold':th, 'precision':round(precision,3), 'recall':round(recall,3), 'fscore':round(fscore,3), 'micro_fscore':round(micro_fscore,3), 'map':round(map_,3)}, ignore_index=True)

            df_results.sort_values(by='fscore', ascending=False).to_csv(abspath + "experiments/" + testing_corpus + "classifier_" + testing_scenario + "train_" + classifier + "_" + type_of_training_data + ".tsv", sep="\t")

        # -------------------------------------------------
        # Apply to test:
        
        # Read stored best parameters:
        parameters_best = pd.read_csv(abspath + "experiments/" + testing_corpus + "classifier_" + testing_scenario + "train_" + classifier + "_" + type_of_training_data + ".tsv", sep="\t").iloc[0]
        obs_threshold = parameters_best['threshold']

        # Apply classifier:
        y_pred = classifiers.classify(abspath,classifier,dataset_testdf[type_of_training_data],type_of_training_data,classifier_corpus,classifier_scenario)
        y_true = [x for x in dataset_testdf['animated'].tolist()]

        # Evaluate:
        precision, recall, fscore, micro_fscore,map_ = animacy_evaluation.results(y_true,y_pred,obs_threshold)
        print("Results:")
        print(type_of_training_data, classifier, "(t=" + str(round(obs_threshold,2)) + ") & " + str(round(precision,3)) + " & " + str(round(recall,3)) + " & " + str(round(fscore,3)) + " & " + str(round(map_,3)))